# Lotka Volterra UPINN

In [1]:
import torch
import sys

# Set the seed for reproducibility
torch.manual_seed(42)

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../')

from utils.BVP import BVP
from utils.UPINN import UPINN
from utils.DataGenerators import LotkaVolterra, sample_with_noise
from utils.Architectures import FNN

Consider the Lotka-Volterra equations, which describe the dynamics of a predator-prey system:

\begin{align}
\frac{dx}{dt} &= \alpha x - \beta x y, \\
\frac{dy}{dt} &= - \delta y + \gamma x y,
\end{align}

where $x$ is the number of prey, $y$ is the number of predators, and $\alpha$, $\beta$, $\gamma$, and $\delta$ are positive constants.

## Generate Data from System

In [2]:
###############################################
### Generate data from Lotka-Volterra model ###
###############################################
###   dx/dt = alpha*x - beta*x*y            ###
###   dy/dt = gamma*x*y - delta*y           ###
###############################################
alpha, beta, gamma, delta = 2/3, 4/3, 1.0, 1.0
x0, y0 = 1.0, 1.0
LV = LotkaVolterra(alpha, beta, gamma, delta, torch.tensor([x0, y0], dtype=torch.float32))

time_int = [0, 25]
train_test = 0.8
N = 800
t = torch.linspace(time_int[0], time_int[1], N)
X = LV.solve(t)
train_idx = torch.arange(0, train_test*N, dtype=torch.long)
test_idx = torch.arange(train_test*N, N, dtype=torch.long)

# Sample subset and add noise
t_d, X_d = sample_with_noise(10, t[train_idx], X[train_idx], epsilon=5e-3)

### Setup Boundary Value Problem

Assume that $\gamma x y$ is not known, and we want to learn it from data.

In [3]:
class LV_BVP(BVP):
    
    def __init__(self, params):
        super().__init__(params)

    def f(self, z, U):
        alpha = self.params['alpha'] if 'alpha' in self.params else self.alpha
        beta = self.params['beta'] if 'beta' in self.params else self.beta
        delta = self.params['delta'] if 'delta' in self.params else self.delta

        dUdt = torch.cat([
        torch.autograd.grad(outputs=U[:, i], inputs=z, grad_outputs=torch.ones_like(U[:, i]), create_graph=True)[0]
        for i in range(U.shape[1])
        ], dim=-1)

        return torch.stack([
            dUdt[:, 0] - alpha*U[:, 0] + beta*U[:, 0]*U[:, 1],
            dUdt[:, 1] + delta*U[:, 1] # - gamma*U[:, 0]*U[:, 1] <-- Estimate this
        ], dim=-1)
    

    def g(self, z, U):
        return U - torch.tensor([1.0, 1.0], device=U.device)


# Define model
# params = dict(
#     alpha=torch.nn.Parameter(torch.tensor(0.5)),
#     beta=torch.nn.Parameter(torch.tensor(0.5)),
#     delta=torch.nn.Parameter(torch.tensor(0.5))
# )

params = dict(
    alpha=alpha,
    beta=torch.nn.Parameter(torch.tensor(1.0)),
    delta=delta,
)
bvp = LV_BVP(params)

### Setup UPINN

In [4]:
# Define model architectures
hidden = [16] * 4
u = FNN(
    dims=[1, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Softplus(),
)
G = FNN(
    dims=[2, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.ReLU(),
)

# Setup scaling layer
u.scale_fn = lambda t_: (t_-t.min())/(t.max()-t.min())
mu, sigma = 0, 2
epsilon = 1e-8
G.scale_fn = lambda x: (x-mu)/(sigma+epsilon)


upinn = UPINN(u, G, bvp=bvp)

Make a class that can plot the results of the UPINN.

In [5]:
import plotly.graph_objects as go

class LV_Plotter:
    def __init__(self, t, X, t_d, X_d, gamma):
        self.t = t
        self.X = X
        self.z_d = t_d
        self.U_d = X_d
        self.gamma = gamma

    def __call__(self, u, G):
        device = next(u.parameters()).device  # Get the device of the model
        plots = {}
        set_free = lambda x: x.squeeze().cpu().detach().numpy()

        # Evaluate the model
        X_pred = u(self.t.unsqueeze(-1).to(device))

        # Plot with Plotly
        fig = go.Figure()
        for i, color, label in zip([0, 1], ['green', 'red'], ['Prey', 'Predator']):
            fig.add_scatter(x=set_free(self.t), y=set_free(self.X[:, i]), mode='lines', name=f'{label}', line=dict(dash='dash', color=color))
            fig.add_scatter(x=set_free(self.t), y=set_free(X_pred[:, i]), mode='lines', name=f'{label} (pred)', line=dict(color=color))
            fig.add_scatter(x=set_free(self.z_d), y=set_free(self.U_d[:, i]), mode='markers', name=f'{label} (data)', marker=dict(color=color, symbol='x'))
        
        fig.update_layout(title="Lotka-Volterra Model")
        plots["Solution"] = fig

        # Plot missing terms
        res = G(X_pred).cpu()
        true_res_dy = (self.gamma * self.X[:, 0] * self.X[:, 1]).cpu().numpy()
        true_res_dx = torch.zeros_like(res[:, 0])

        fig = go.Figure()
        for res_val, label, color in zip([res[:, 0], res[:, 1]], ['Residual Prey', 'Residual Predator'], ['green', 'red']):
            fig.add_scatter(x=set_free(self.t), y=set_free(res_val), mode='lines', name=label, line=dict(color=color))
        fig.add_scatter(x=set_free(self.t), y=true_res_dx, mode='lines', name='Prey: 0', line=dict(dash='dash', color='green'))
        fig.add_scatter(x=set_free(self.t), y=true_res_dy, mode='lines', name='Predator: γ*x*y', line=dict(dash='dash', color='red'))
        fig.update_layout(title="Lotka-Volterra Missing Terms")

        plots["Missing Terms"] = fig
        return plots

plotter = LV_Plotter(t, X, t_d, X_d, gamma=gamma)

## Train UPINN

In [6]:
upinn.train(
    data_points=t_d.unsqueeze(-1),
    data_target=X_d,
    boundary_points=torch.tensor([[0.0]]),
    collocation_points=t[train_idx].unsqueeze(-1).requires_grad_(True),
    epochs=10000,
    # log_wandb=dict(name='UPINN', project='Master-Thesis', plotter=plotter, plot_interval=1000),
    log_wandb=None,
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=1e-2, weight_decay=1e-10),
    beta_softadapt=0.1,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
    scheduler_args=dict(factor=0.5, patience=100, min_lr=1e-6),
    loss_tol=1e-5,
    )

Beginning training...
Running on: cuda


 36%|███▌      | 3610/10000 [00:50<01:29, 71.71it/s, Loss=1e-5, BC Loss=3.3e-7, PDE Loss=2.35e-5, Data Loss=6.14e-6]        

Loss below tolerance at epoch 3610. Terminating training.
Training complete.


In [7]:
plots = plotter(upinn.u, upinn.G)
plots["Solution"].show()
plots["Missing Terms"].show()

## Train beyond the data

In [13]:
# Freeze G and parameters
G.requires_grad_(False);
bvp.requires_grad_(False);

# Extend collocation points to include the entire time interval and train again
upinn.train(
    data_points=t_d.unsqueeze(-1),
    data_target=X_d,
    boundary_points=torch.tensor([[0.0]]),
    collocation_points=t.unsqueeze(-1).requires_grad_(True),
    epochs=10000,
    log_wandb=None,
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=3e-4, weight_decay=1e-10),
    beta_softadapt=0.1,
    loss_tol=1e-6,
    )

Beginning training...
Running on: cuda


100%|██████████| 10000/10000 [02:12<00:00, 75.23it/s, Loss=6.3e-5, BC Loss=5.16e-6, PDE Loss=0.000114, Data Loss=7.01e-5]   

Training complete.


In [14]:
plots = plotter(upinn.u, upinn.G)
plots["Solution"].show()
plots["Missing Terms"].show()